In [45]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
from itertools import compress
import time
import os

In [46]:
options = {'driver':r'C:\Users\likith\Desktop\QGIS\project\chromedriver.exe',
           'output':'CPCB_Data',
           'sdate':'01-01-2019'
          }

In [47]:
global url,option,browser
url = 'https://app.cpcbccr.com/AQI_India/#' 
option=webdriver.ChromeOptions()
browser = webdriver.Chrome(executable_path="{}".format(options['driver']), chrome_options=option)
browser.get(url)
time.sleep(5)

<ipython-input-47-029efec1e6d7>:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path="{}".format(options['driver']), chrome_options=option)


In [48]:
def get_states():
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table = soup.find("select",{"id":"states"})
    states  = []
    for i in table.find_all("option"):
        states.append(i.text)
    return states[1:]

In [49]:
def get_cities(states,state):
    path = '//*[@id="states"]/option['+str(states.index(state)+2)+']'
    browser.find_element_by_xpath(path).click()
    time.sleep(5)
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table2 = soup.find("select",{"id":"cities"})
    cities  = []
    for i in table2.find_all("option"):
        cities.append(i.text)
    return cities[1:]

In [50]:
def get_stations(cities,city):
    path = '//*[@id="cities"]/option['+str(cities.index(city)+2)+']'
    browser.find_element_by_xpath(path).click()
    time.sleep(5)
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table2 = soup.find("select",{"id":"stations"})
    stations  = []
    for i in table2.find_all("option"):
        stations.append(i.text)
    return stations[1:]

In [69]:
def extract_data(station,date):
    date = date
    station=station
    path = '//*[@id="date"]/input'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).clear();
    browser.find_element_by_xpath(path).send_keys(date)
    time.sleep(2)
    over_data = []
    path = '//*[@id="date"]/input'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).clear()
    time.sleep(2)
    browser.find_element_by_xpath(path).send_keys(date)
    time.sleep(2)
    browser.find_element_by_xpath('/html/body').click()
    path = '//*[@id="time"]'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).clear()
    time.sleep(2)
    browser.find_element_by_xpath(path).send_keys('0:00')
    browser.find_element_by_xpath('/html/body').click()
    time.sleep(2)
    data =[]
    s = [station]
    path = '//*[@id="stations"]/option['+str(4+2)+']'
    browser.find_element_by_xpath(path).click()
    time.sleep(5)
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    tab =soup.find_all('td',{"class":"element-name"})
    parameters  = []
    for i in tab:
        parameters.append(i.text)
    req = ['PM2.5','PM10']
#     req = ['PM2.5']

    tab =soup.find_all('div',{"aria-label":"A tabular representation of the data in the chart."})
    try:
        count = 0
        date = []
        pm = []
        x =tab[parameters.index(req[0])].find_all('td')
        y =tab[parameters.index(req[1])].find_all('td')
        for j in x:
            if count%2 == 1:
                pm.append(j.text)
            else:
                date.append(j.text)
            count = count+1
        data.append(list(s))
        data.append(list(pm))
        data.append(list(date))
        count = 0
        date = []
        pm = []
        for j in y:
            if count%2 == 1:
                pm.append(j.text)
            else:
                date.append(j.text)
            count = count+1
        data.append(list(pm))
        data.append(list(date))
    except:
        pass
    if len(data) >0:
        over_data.append(data)
    return over_data

In [52]:
cities = []
states = get_states()
print(states)
inp = input('Enter the state name here : ')
if inp != 'All':
    cities = get_cities(states,inp)
else:
    for i in states:
        print(get_cities(states,i))

['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Delhi', 'Gujarat', 'Haryana', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Meghalaya ', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'West Bengal']
Enter the state name here : Karnataka


In [53]:
print(cities)
stations = []
inp = input('Enter the city name here :   ')
stations = get_stations(cities,inp)

['Bagalkot', 'Bengaluru', 'Bidar', 'Chamarajanagar', 'Chikkaballapur', 'Chikkamagaluru', 'Davanagere', 'Gadag', 'Hassan', 'Hubballi', 'Kalaburagi', 'Kolar', 'Koppal', 'Madikeri', 'Mangalore', 'Mysuru', 'Raichur', 'Ramanagara', 'Shivamogga', 'Udupi', 'Vijayapura', 'Yadgir']
Enter the city name here :   Bengaluru


In [54]:
stations

['BTM Layout, Bengaluru - CPCB',
 'BWSSB Kadabesanahalli, Bengaluru - CPCB',
 'Bapuji Nagar, Bengaluru - KSPCB',
 'City Railway Station, Bengaluru - KSPCB',
 'Hebbal, Bengaluru - KSPCB',
 'Hombegowda Nagar, Bengaluru - KSPCB',
 'Jayanagar 5th Block, Bengaluru - KSPCB',
 'Peenya, Bengaluru - CPCB',
 'Sanegurava Halli, Bengaluru - KSPCB',
 'Silk Board, Bengaluru - KSPCB']

In [61]:
def create_dataframe(over_data):
    stati = []
    p = []
    d = []
    p2 = []
    for i in dta:
        for j in range(len(i[2])):
            stati.append(i[0][0][0])
        for k in range(len(i[1])):
            p.append(i[1][k])
            d.append(i[2][k])
            p2.append(i[3][k])
    return (stati,p,d,p2)

In [56]:
import datetime
d1 = datetime.date(2020, 1, 1)
d2 = datetime.date(2020, 12, 31)
days = [d1 + datetime.timedelta(days=x) for x in range((d2-d1).days + 1)]
req_dates = []
for day in days:
    req_dates.append(day.strftime('%d/%m/%Y'))

In [70]:
import pandas as pd
df = pd.DataFrame(columns = ['station','time','PM2.5','PM10'])

In [71]:
for i in req_dates:
    dta = extract_data(['Hebbal, Bengaluru - KSPCB'],i)
    stati,p,d,p2 = create_dataframe(dta)
    for j in range(len(stati)):
        df = df.append({'station':stati[j] , 'time' : d[j] , 'PM2.5':p[j],'PM10':p2[j]},ignore_index=True)
#         df['station'] = stati
#         df['PM2.5']=p
#         df['time'] = d
#     else:
#         stati,p,d = create_dataframe(dta)
#         df['station'].append(stati)
#         df['PM2.5'].append(p)
#         df['time'].append(d)

In [72]:
df.to_csv('HEBBAL_2020.csv')

In [36]:
stati = []
p = []
d = []
p2 = []
for i in dta:
    for j in range(len(i[2])):
        stati.append(i[0][0])
    for k in range(len(i[1])):
        p.append(i[1][k])
        d.append(i[2][k])
        p2.append(i[3][k])

In [37]:
for j in range(len(stati)):
    df = df.append({'station':stati[j] , 'time' : d[j] , 'PM2.5':p[j],'PM10':p2[j]},ignore_index=True)

In [38]:
df.head(50)

,station,time,PM2.5,PM10
0,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 1:00:00 AM",91,92
1,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 2:00:00 AM",121,109
2,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 3:00:00 AM",79,93
3,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 4:00:00 AM",70,81
4,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 5:00:00 AM",61,66
5,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 6:00:00 AM",50,61
6,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 7:00:00 AM",69,75
7,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 8:00:00 AM",80,77
8,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 9:00:00 AM",98,100
9,"[Hebbal, Bengaluru - KSPCB]","Jan 1, 2020, 10:00:00 AM",96,106


In [3]:
import pandas as pd
df = pd.read_csv("HEBBAL_2020.csv")
df.head(50)

,Unnamed: 0,station,time,PM2.5,PM10
0,0,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 1:00:00 AM",73.0,85.0
1,1,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 2:00:00 AM",76.0,82.0
2,2,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 3:00:00 AM",71.0,80.0
3,3,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 4:00:00 AM",72.0,80.0
4,4,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 5:00:00 AM",73.0,77.0
5,5,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 6:00:00 AM",88.0,78.0
6,6,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 7:00:00 AM",88.0,85.0
7,7,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 8:00:00 AM",104.0,104.0
8,8,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 9:00:00 AM",135.0,120.0
9,9,"Hebbal, Bengaluru - KSPCB","Dec 31, 2019, 10:00:00 AM",95.0,103.0


In [6]:
date = []
for i in df['time']:
    date.append((i.replace(i.split(",")[-1], "")).replace(",",""))

In [8]:
df['date'] = date

In [17]:
df.head()

,Unnamed: 0,station,PM2.5,PM10,date
0,0,"Hebbal, Bengaluru - KSPCB",73.0,85.0,Dec 31 2019
1,1,"Hebbal, Bengaluru - KSPCB",76.0,82.0,Dec 31 2019
2,2,"Hebbal, Bengaluru - KSPCB",71.0,80.0,Dec 31 2019
3,3,"Hebbal, Bengaluru - KSPCB",72.0,80.0,Dec 31 2019
4,4,"Hebbal, Bengaluru - KSPCB",73.0,77.0,Dec 31 2019


In [47]:
df = df.dropna()

In [12]:
df = df.drop(['time'],axis=1)

In [26]:
df = df.drop(['station'],axis=1)


In [28]:
df = df.drop(['Unnamed: 0'],axis=1)

In [41]:
df['PM2.5'][0:23].mean()

81.34782608695652

In [30]:
len(df['date'].unique())

355

In [48]:
x = df.groupby(['date'],sort=False).mean()

In [49]:
x.to_csv("edit_Hebbal_2020.csv")